# week 8

Q1

In [2]:
import pandas as pd

# Load
df1 = pd.read_csv("homework_8.1.csv")
df1.head()

,Unnamed: 0,X,Y,Z
0,0,1,4.109218,1.764052
1,1,0,2.259504,0.400157
2,2,0,-0.647584,0.978738
3,3,0,2.106071,2.240893
4,4,1,3.583464,1.867558


In [3]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# 1. Estimate propensity scores using logistic regression (Z predicts X)
logit = LogisticRegression()
logit.fit(df1[['Z']], df1['X'])

# predict propensity scores
df1['propensity'] = logit.predict_proba(df1[['Z']])[:, 1]

# 2. Find inverse probability weights
df1['weight'] = np.where(df1['X'] == 1, 1 / df1['propensity'], 1 / (1 - df1['propensity']))

# 3. Calc weighted averages of Y for treated and control
treated = df1[df1['X'] == 1]
control = df1[df1['X'] == 0]

weighted_mean_treated = np.sum(treated['Y'] * treated['weight']) / np.sum(treated['weight'])
weighted_mean_control = np.sum(control['Y'] * control['weight']) / np.sum(control['weight'])

# 4. Estimate ATE
ate_ipw = weighted_mean_treated - weighted_mean_control
ate_ipw


np.float64(2.2743411898510133)

Q2

In [5]:
# First three propensity scores
df1['propensity'].head(3).tolist()

[0.840113713150768, 0.5846459703178026, 0.711082447582062]

Q3

In [7]:
from scipy.spatial.distance import mahalanobis

# Load
df2 = pd.read_csv("homework_8.2.csv")
df2.head()

,Unnamed: 0,X,Y,Z1,Z2
0,0,1,4.652085,1.764052,2.320015
1,1,1,2.590221,0.400157,1.292631
2,2,1,3.898981,0.978738,0.556423
3,3,1,5.857179,2.240893,2.345607
4,4,1,3.647489,1.867558,2.095611


In [8]:
# 1. Split reated and untreated
treated = df2[df2['X'] == 1].reset_index(drop=True)
control = df2[df2['X'] == 0].reset_index(drop=True)

# 2. Covariance matrix using Z1 and Z2
Z = df2[['Z1', 'Z2']].values
cov = np.cov(Z, rowvar=False)
inv_cov = np.linalg.inv(cov)

# 3. For each treated, find closest control via Mahalanobis distance
matches = []
for i in range(len(treated)):
    z_treat = treated.loc[i, ['Z1', 'Z2']].values
    dists = control[['Z1', 'Z2']].apply(
        lambda row: mahalanobis(z_treat, row.values, inv_cov),
        axis=1
    )
    nearest_idx = dists.idxmin()  # index of nearest untreated
    matches.append(control.loc[nearest_idx])

# 4. Convert matches into DataFrame
matched_controls = pd.DataFrame(matches).reset_index(drop=True)

# 5. Compute ATE
ate = np.mean(treated['Y'].values - matched_controls['Y'].values)
print("ATE (Mahalanobis Matching):", ate)

ATE (Mahalanobis Matching): 3.4376789979126094


Q4

In [10]:
# 1. Split into treated and control
treated = df2[df2['X'] == 1].reset_index(drop=True)
control = df2[df2['X'] == 0].reset_index(drop=True)

# 2. Inverse covariance of Z1 and Z2
Z = df2[['Z1', 'Z2']].values
cov = np.cov(Z, rowvar=False)
inv_cov = np.linalg.inv(cov)

# 3. Find the nearest control for each treated and record distance
distances = []
for i in range(len(treated)):
    z_treat = treated.loc[i, ['Z1', 'Z2']].values
    dists = control[['Z1', 'Z2']].apply(
        lambda row: mahalanobis(z_treat, row.values, inv_cov),
        axis=1
    )
    nearest_idx = dists.idxmin()
    min_dist = dists.min()
    distances.append((i, min_dist, nearest_idx))

# 4. Find treated item with the biggest/farthest min distance
farthest_match = max(distances, key=lambda x: x[1])
treated_idx, farthest_dist, nearest_control_idx = farthest_match

# 5. Extract those rows
farthest_treated = treated.loc[treated_idx, ['Z1', 'Z2']]
nearest_control = control.loc[nearest_control_idx, ['Z1', 'Z2']]

print("Treated item (least common support):")
print(farthest_treated)
print("\nNearest untreated match:")
print(nearest_control)
print(f"\nMahalanobis distance: {farthest_dist:.3f}")

Treated item (least common support):
Z1    2.696224
Z2    0.538155
Name: 241, dtype: float64

Nearest untreated match:
Z1    1.519995
Z2   -1.282208
Name: 217, dtype: float64

Mahalanobis distance: 1.383
